# CLIP transfer learning
---
## Introduction

In this notebook we will look at how to fine-tune CLIP models for the [Oxford-pets](https://huggingface.co/datasets/pcuenq/oxford-pets) dataset.

[CLIP](https://arxiv.org/abs/2103.00020), or Contrastive Language-Image Pre-training, is a multimodal model that combines language and vision to extract features from text and images. The resulting features are then projected into a common Euclidean space, allowing for meaningful comparisons between text and image features using the dot product.

Normally, CLIP operates in zero-shot prediction mode or functions as a feature extractor. Consequently, fine-tuning CLIP diverges from the usual process, as seen in tasks like fine-tuning ResNet for classification. In our scenario, the model must acquire the most valuable features from both the input text and image data.

Evaluating the quality of the features extracted remains an ongoing area of active research. One method employed for this assessment involves utilizing the extracted features to address the classification problem, either through the application of a [linear classifier probe](https://h2o.ai/wiki/probing-classifiers/) or adopting a zero-shot approach.

---

Let's start with importing necessary functions, libraries and environmental variables.

In [1]:
!pip install evaluate

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


In [2]:
import os
import random
from functools import partial
from typing import Any
import evaluate
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from datasets import Dataset, DatasetDict, load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import (CLIPImageProcessor, CLIPModel, CLIPProcessor,
                          CLIPTokenizerFast, Trainer, TrainingArguments)
from transformers.tokenization_utils_base import BatchEncoding
from datasets.formatting.formatting import LazyBatch

In [3]:
os.environ["CURL_CA_BUNDLE"] = ""
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def seed_all(seed: int):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)

seed_all(69)

In [7]:
!curl -sSL -o /etc/network_turbo https://oss-high-qy01.cdsgss.com/public-static-prod-gpuzoom/installation/platform/cn-northwest1-network_turbo
!chmod +x /etc/network_turbo
!source /etc/network_turbo

学术资源加速设置成功
注意：仅限于学术用途，不承诺稳定性保证


### Exploratory Data Analysis on the Dataset

We'll utilize the [🤗datasets](https://huggingface.co/docs/datasets/index) library to load the [Oxford-pets](https://huggingface.co/datasets/pcuenq/oxford-pets) dataset, comprising 37 pet categories (primarily dogs and cats) with a total of 7,390 images.

Since the dataset exclusively consists of the training portion, we'll proceed to partition it into training, validation, and test sets.

In [ ]:
dataset = load_dataset("pcuenq/oxford-pets")
dataset_train_val = dataset['train'].train_test_split(test_size=0.3)
dataset_val_test = dataset_train_val['test'].train_test_split(test_size=0.2)

dataset = DatasetDict(
    {
        "train": dataset_train_val['train'],
        "val": dataset_val_test['test'],
        "test": dataset_val_test['train']
    }
)

In [ ]:
dataset

An individual item in the dataset consists of the following attributes:

- **path**: A string representing the image file path.
- **label**: A string denoting the pet category.
- **dog**: A boolean value; True if the image features a dog.
- **image**: A PIL image representing the dataset image.

However, for our purposes, we will exclusively utilize the **label** and **image** columns.

In [ ]:
dataset['train'][0]

Now, let's examine a selection of images along with their corresponding labels.








In [ ]:
def show_images(ds: Dataset):
    fig, axs = plt.subplots(3, 3, figsize=(13, 13))
    for i in range(9):
        axs[i // 3][i % 3].imshow(ds[i]['image'])
        axs[i // 3][i % 3].set_title(ds[i]['label'])

show_images(dataset['train'])

For fine-tuning we need process labels to ids.

In [ ]:
labels = set(dataset['train']['label'])
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

labels = list(label2id)

After that we get mapping from label to id.

In [ ]:
label2id

### Dataset preparation

For the default CLIP model, we will use the pre-trained [model by OpenAI](https://huggingface.co/openai/clip-vit-base-patch32). However, feel free to modify the `MODEL_NAME` variable to another model. For instance [this](https://huggingface.co/openai/clip-vit-large-patch14) model of CLIP have a larger number of parameters.

In [ ]:
MODEL_NAME = "openai/clip-vit-base-patch32"

For selected `MODEL_NAME` we need to import corresponding:
- **Tokenizer**: used to encode input text.
- **Image processor**: It is utilized to preprocess input images, handling tasks such as scaling, resizing, and more.

In [ ]:
TOKENIZER = CLIPTokenizerFast.from_pretrained(MODEL_NAME)
IMAGE_PROCESSOR = CLIPImageProcessor.from_pretrained(MODEL_NAME)

Dataset preparation consists of 2 steps
1) **Label Encoding with Tokenizer**: This step entails encoding the label of each dataset item using the tokenizer. Importantly, we encode not only the label's text but also include a special prompt with the label. This enhancement is designed to elevate the quality of text features in the language model.

2) **Image Preparations with CLIP's Image Processor**: In this step, we prepare the images using CLIP's image processor. This involves preprocessing steps to ensure compatibility with CLIP's model architecture.

In [ ]:
def transform_class_labels(
    items: LazyBatch, tokenizer: CLIPTokenizerFast, label2id: dict[str, int]
) -> dict[str, Any]:
    """Function used to encode item's label prompt with tokenizer

    Args:
        items (LazyBatch): Input dataset items
        tokenizer (CLIPTokenizerFast): CLIP's tokenizer
        label2id (dict[str, int]): mapping from label to id

    Returns:
        dict[str, Any]: transformed items
    """
    # this prompt is better than prompt consisting only of the class name
    label_prompt = [f"a photo of {label}" for label in items["label"]]
    output = tokenizer(label_prompt, padding=True, return_tensors="pt")
    items["input_ids"] = output["input_ids"]
    items["attention_mask"] = output["attention_mask"]
    items["label_id"] = [label2id[label] for label in items["label"]]
    return items


def transform_image(items: LazyBatch, image_processor: CLIPImageProcessor) -> dict[str, Any]:
    """Function used to preprocess input image with image processor

    Args:
        items (LazyBatch): Input dataset items
        image_processor (CLIPImageProcessor): CLIP's image processor

    Returns:
        dict[str, Any]: transformed items
    """
    output = image_processor(items["image"], return_tensors="pt")
    items["pixel_values"] = output["pixel_values"]
    return items

We can efficiently apply `transform_class_labels` function on each dataset split with [map function](https://huggingface.co/docs/datasets/process#map) inplace.

But for `transform_image` function we'll use [set_transform](https://huggingface.co/docs/datasets/v2.16.1/en/package_reference/main_classes#datasets.Dataset.set_transform) to lazily process pictures because it's inefficient to store them in memory

In [ ]:
dataset = dataset.map(partial(transform_class_labels, tokenizer=TOKENIZER, label2id=label2id), batched=True)
dataset.set_transform(partial(transform_image, image_processor=IMAGE_PROCESSOR))

### CLIP Image Classification

In [ ]:
# utilities functions
def get_module_device(module: nn.Module) -> torch.device:
    """Function to get current module device

    Args:
        module (nn.Module): pytorch module

    Returns:
        torch.device: module's device
    """
    return next(module.parameters()).device


def freeze_params(module: nn.Module, freeze_top_percent: float = 1.0) -> None:
    """Function used to freeze module by setting 'requires_grad' to False

    Args:
        module (nn.Module): pytorch module
        freeze_top_percent (float, optional): percentage of layers starting from the top that need to be frozen. Defaults to 1.0.
    """
    all_params_length = len(list(module.parameters()))
    for indx, param in enumerate(module.parameters()):
        if int(all_params_length * freeze_top_percent) <= indx:
            break
        param.requires_grad = False


def print_trainable_parameters(model: nn.Module) -> None:
    """Function prints statistics about trainable parameters

    Args:
        model (nn.Module): pytorch module
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params: {(trainable_params / 10**6):.4f}M || All params: {(all_param / 10**6):.4f}M || Trainable%: {100 * trainable_params / all_param:.2f}%"
    )

Our CLIP based classification model will work as follows:
1. **Prompt Generation**: Initial prompts are generated based on the labels present in the dataset.
2. **Feature Extraction with Language CLIP Model**: Features are extracted from these prompts using a language CLIP model.
3. **Visual Feature Extraction**: For each image, features are extracted using a visual CLIP model.
4. **Similarity Score Computation**: A similarity score between the image and the set of prompts is computed.
5. **Prediction Based on Similarity**: The most similar prompt is selected, and its associated label is used as the final prediction for the image.

In [ ]:
class CLIPClassifier(nn.Module):
    def __init__(self, clip_model: CLIPModel, tokenizer: CLIPTokenizerFast, labels: list[str]):
        super().__init__()
        self.model = clip_model
        self.tokenizer = tokenizer
        self.logit_scale = self.model.logit_scale.exp()
        self.label2id = {label: i for i, label in enumerate(labels)}
        self.labels_embeddings = nn.Parameter(self.generate_labels_embeddings(labels))

    def generate_labels_embeddings(self, labels: list[str]) -> torch.Tensor:
        """Function prepares label's prompt embeddings usings language model

        Args:
            labels (list[str]): list with dataset labels

        Returns:
            torch.Tensor: labels embeddings of size (num_labels, dim)
        """
        # tokenize label's prompt
        labels_inputs = self.tokenizer(
            [f"a photo of {label}" for label in labels],
            return_tensors="pt",
            padding=True,
        ).to(get_module_device(self.model))
        # run language model on this prompt and get embeddings
        labels_embeddings = self.model.get_text_features(**labels_inputs)
        # normilize embeddings
        labels_embeddings /= labels_embeddings.norm(p=2, dim=-1, keepdim=True)
        return labels_embeddings

    def forward(self, images: torch.Tensor) -> torch.Tensor:
        """Function to calculate similarity between image and label's promts embeddings

        Args:
            images (torch.Tensor): image tensor

        Returns:
            torch.Tensor: output similarity tensor of size (batch_size,  num_labels)
        """
        image_features = self.model.get_image_features(
            images
        )  # (batch_size, dim)
        image_features /= image_features.norm(p=2, dim=-1, keepdim=True)
        return torch.matmul(image_features, self.labels_embeddings.T) * self.logit_scale

def calculate_accuracy(model: CLIPClassifier, dataloader: DataLoader) -> float:
    """Function calculates accuracy of given model on dataloader

    Args:
        model (CLIPClassifier): CLIP classifier model
        dataloader (DataLoader): evaluation dataloader

    Returns:
        float: model's accuracy
    """
    metric = evaluate.load("accuracy")
    predictions_list = []
    references_list = []
    device = get_module_device(model)
    for batch in tqdm(
        dataloader, total=len(dataloader), desc="Evaluate model on dataset"
    ):
        batch["pixel_values"] = batch["pixel_values"].to(device)
        predictions = model(batch["pixel_values"])
        predictions_list.append(torch.argmax(predictions, dim=1))
        references_list.append(batch["label_id"])
    return metric.compute(
        predictions=torch.concat(predictions_list),
        references=torch.concat(references_list),
    )["accuracy"]

def collate_fn(items: LazyBatch) -> dict[str, Any]:
    return {
        "pixel_values": torch.stack(
            [item["pixel_values"] for item in items]
        ),  # (batch_size, 3, 224, 224)
        "input_ids": torch.tensor(
            [item["input_ids"] for item in items]
        ),  # (batch_size, max_length)
        "attention_mask": torch.tensor(
            [item["attention_mask"] for item in items]
        ),  # (batch_size, max_length)
        "label_id": torch.tensor([item["label_id"] for item in items]),  # (batch_size),
        "return_loss": True,
    }

@torch.no_grad()
def evaluate_clip_classifier(
    model: nn.Module,
    dataset: Dataset,
    tokenizer: CLIPTokenizerFast,
    labels: list[str],
    batch_size: int = 64,
    num_workers: int = 5,
    device: str = "cuda",
) -> None:
    """Function evaluates CLIP model on given dataset

    Args:
        model (nn.Module): CLIP model
        dataset (Dataset): evaluation dataset
        tokenizer (CLIPTokenizerFast): CLIP tokenizer
        labels (list[str]): list with dataset labels
        batch_size (int): batch size. Defaults to 64
        num_workers (int): number of workers for dataloader. Defaults to 5
        device (str, optional): model's device. Defaults to "cuda".
    """
    clip_classifier = CLIPClassifier(model, tokenizer, labels)
    test_dataloader = DataLoader(
        dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=collate_fn
    )
    clip_classifier = clip_classifier.to(device)
    acc = calculate_accuracy(clip_classifier, test_dataloader)
    print(f"Model accuracy: {acc}")

### Evaluate baseline CLIP (Zero-shot)

First, let's evaluate our pretrained CLIP model on test dataset. One of the advantages of CLIP model compared to standard classification models is that it works well without fine-tuning.

In [ ]:
clip_baseline = CLIPModel.from_pretrained(MODEL_NAME)
freeze_params(clip_baseline)
print_trainable_parameters(clip_baseline)
evaluate_clip_classifier(clip_baseline, dataset['test'], TOKENIZER, labels)

In our case we got 0.8 accuracy for free! But we can improve it even further our metrics with fine-tuning techniques.

## Fine-tuning

In this section, we'll undertake the fine-tuning process leveraging the [🤗HuggingFace Trainer API](https://huggingface.co/docs/transformers/main_classes/trainer). The Trainer API significantly streamlines the training code, eliminating much of the boilerplate, and offers various features such as the ability to save model weights and training in fp16.

In [ ]:
def collate_train_fn(items: LazyBatch):
    items = collate_fn(items)
    items.pop("label_id")
    return items


def get_default_training_args(
    experiment_name: str,
    lr: float,
    batch_size: int = 256,
    num_epoch: int = 2,
    num_workers: int = 15,
) -> TrainingArguments:
    """Function gets default training arguments for HF Trainer

    Args:
        experiment_name (str): experiment name
        lr (float): learning rate
        batch_size (int, optional): batch size. Defaults to 256.
        num_epoch (int, optional): number of epoch to train. Defaults to 2.
        num_workers (int, optional): number of workers for dataloader. Defaults to 15.

    Returns:
        TrainingArguments: arguments object for HF Trainer
    """
    return TrainingArguments(
        experiment_name,
        per_device_train_batch_size=batch_size,
        learning_rate=lr,
        num_train_epochs=num_epoch,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=1,
        logging_steps=10,
        save_total_limit=2,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        fp16=True,
        remove_unused_columns=False,
        load_best_model_at_end=True,
        dataloader_num_workers=num_workers,
    )

### All layers tuning

Initially, let's explore the simplest approach to fine-tuning, which involves fine-tuning all layers of the model. While straightforward, it's important to note that this method is computationally intensive.

In [ ]:
clip_full_finetuned = CLIPModel.from_pretrained(MODEL_NAME)
trainer = Trainer(
    model=clip_full_finetuned,
    args=get_default_training_args("clip-all-layers-tuning-oxford-pets", 3e-6),
    data_collator=collate_train_fn,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
)

trainer.train()

In [ ]:
print_trainable_parameters(clip_full_finetuned)
evaluate_clip_classifier(clip_full_finetuned, dataset['test'], TOKENIZER, labels)

### Fine-tuning Text Encoder

Following that, we have the option to fine-tune only the language model while keeping the visual model weights frozen. This method proves beneficial when dataset labels are provided in a language that differs from the one in which the language model was originally trained.

In [ ]:
clip_text_model_tuning = CLIPModel.from_pretrained(MODEL_NAME)
freeze_params(clip_text_model_tuning.vision_model)

trainer = Trainer(
    model=clip_text_model_tuning,
    args=get_default_training_args("clip-text-model-tuning-oxford-pets", 3e-5),
    data_collator=collate_train_fn,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
)

trainer.train()

In [ ]:
print_trainable_parameters(clip_text_model_tuning)
evaluate_clip_classifier(clip_text_model_tuning, dataset['test'], TOKENIZER, labels)

### Fine-tuning Image Encoder

We can do the opposite and fine-tune only vision model. This can be useful if the image domain in your dataset is quite specific, such as satellite images

In [ ]:
clip_vision_model_tuning = CLIPModel.from_pretrained(MODEL_NAME)
freeze_params(clip_vision_model_tuning.text_model)

trainer = Trainer(
    model=clip_vision_model_tuning,
    args=get_default_training_args("clip-vision-model-tuning-oxford-pets", 3e-5),
    data_collator=collate_train_fn,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

trainer.train()

In [ ]:
print_trainable_parameters(clip_vision_model_tuning)
evaluate_clip_classifier(clip_vision_model_tuning, dataset['test'], TOKENIZER, labels)

### Fine-tune 30 percent of the Last Layers of Vision-Language Model

The most universal option is to fine-tune only the last layers in both models. This approach strikes a balance, allowing adaptation to the specific task while leveraging the pre-trained knowledge from the lower layers.

In [ ]:
clip_partial_tuning = CLIPModel.from_pretrained(MODEL_NAME)
freeze_params(clip_partial_tuning.text_model, freeze_top_percent=0.7)
freeze_params(clip_partial_tuning.vision_model, freeze_top_percent=0.7)

In [ ]:
trainer = Trainer(
    model=clip_partial_tuning,
    args=get_default_training_args("clip-partial-model-tuning-oxford-pets", 3e-5),
    data_collator=collate_train_fn,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
)

trainer.train()

In [ ]:
print_trainable_parameters(clip_partial_tuning)
evaluate_clip_classifier(clip_partial_tuning, dataset['test'], TOKENIZER, labels)

### PEFT tuning (with LoRA)

In cases where you want to fine-tune a pre-trained model with a large number of parameters, you can use the [PEFT](https://huggingface.co/blog/peft)(Parameter-Efficient Fine-Tuning) techniques. PEFT allows you to train a small number of parameters (about 1-5% of all model parameters) with performance comparable to training all layers

In our case, we will use the [LoRA](https://huggingface.co/docs/peft/conceptual_guides/lora) technique using the [🤗HuggingFace PEFT](https://huggingface.co/docs/peft/index) library.

**LoRA** introduces two small-rank matrices (called *update matrices*) to the initial weights of the pretrained model. Throughout the fine-tuning process, the initial weights of the model remain frozen, and only the update matrices are trained.

In our specific case, we'll employ update matrices of rank 64, applied to the query, key, and value matrices in both the visual and language components of the CLIP model.

In [ ]:
clip_lora_tuning = CLIPModel.from_pretrained(MODEL_NAME)
config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=['q_proj', 'k_proj', 'v_proj'],
)

lora_model = get_peft_model(clip_lora_tuning, config)

In [ ]:
trainer = Trainer(
    model=lora_model,
    args=get_default_training_args("clip-lora-model-tuning-oxford-pets", 3e-4),
    data_collator=collate_train_fn,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
)

trainer.train()

In [ ]:
print_trainable_parameters(lora_model)
evaluate_clip_classifier(lora_model, dataset['test'], TOKENIZER, labels)

## Conclusion

In this notebook, we explored the fine-tuning of a CLIP model on the Oxford-pets dataset, assessing the model's performance in zero-shot mode and through various fine-tuning methods, including PEFT.

The achieved results are summarized below:

| Model                       | Trainable Parameters (M) |  Trainable Parameters (%) | Accuracy    |
| --------------------------- | ------------------------ | ------------------------- | ----------- |
| Baseline (Zero-shot)        | 0                        | 0%                        | 0.8003      |
| Full fine-tuning            | 151.28                   | 100%                      | 0.9278      |
| Only language model tuning  | 63.82                    | 42.19%                    | 0.9199      |
| Only vision model tuning    | 88.11                    | 58.24%                    | 0.9250      |
|Top 30% of both models tuning| 38.79                    | 25.65%                    | **0.9334**  |  
| LoRA tuning                 | **5.90**                 | **3.75%**                 | 0.9261      |

In our case, the best result was obtained from the model with fine-tuning only top 30% of layers.

Feel free to experiment with training parameters in the Trainer and CLIP model settings to potentially enhance the results further!